In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.append("../")

In [ ]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "6"
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"

In [ ]:
import pandas as pd
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from transformers import T5Tokenizer

In [ ]:
# [pin]
file_path = "../data/data.csv"
root_path = "../data/"


df = pd.read_csv(file_path)
df["prefix"] = "clsorg"
df = df.rename({"message": "input_text", "label": "target_text"}, axis=1)
df.sample(20)

,input_text,target_text,prefix
7507,????#GLTR #buyback АКЦИОНЕРЫ GLOBALTRANS УТВЕР...,220,clsorg
12541,​НАПРАВЛЕНИЯ ЛОКАЛЬНЫХ ТРЕНДОВ ПО БАЗОВЫМ ИНСТ...,127;150;7;99;152,clsorg
10206,"ГОСА - ФосАгро: ДИВИДЕНДЫ = 465 рублей, ДД: ...",187,clsorg
2446,"""TRYRUB Московская биржа запустила торги валют...",103,clsorg
4161,"""🛢🇷🇺#нефть #газ #спг #россия заявления Алекса...",48,clsorg
9156,"GMKN - RUAL - ОФЕРТА «Русал», подтвердивший го...",11,clsorg
3865,"""🏁 Итоги дня: 24 мая 📦 Ozon раскрыл сильные р...",204;26,clsorg
14394,🇷🇺#SMLT #отчетность Самолет — РСБУ 9м 2023г,56,clsorg
15858,🔍 Взгляд на компанию: ММК – операционные резул...,89;116;90,clsorg
15497,"💸 Мосбиржа объявила дивиденды за 2023 17,35 р...",103,clsorg


In [ ]:
m_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(m_name)

In [ ]:
from src.t5.dataset import NERDataModel
from src.t5.model import NERModel

BATCH_SIZE = 64
EPOCHS = 10
train_df, test_df = train_test_split(df, test_size=0.25, random_state=42)
data_module = NERDataModel(train_df, test_df, tokenizer, batch_size=BATCH_SIZE)
data_module.setup()

In [ ]:
model = NERModel()

checkpoint_callback = ModelCheckpoint(
    dirpath="checkpoints",
    filename="ner",
    save_top_k=1,
    verbose=True,
    monitor="val_loss",
    mode="min",
)

trainer = Trainer(
    callbacks=[checkpoint_callback],
    max_epochs=EPOCHS,
    accelerator="cuda",
)

In [ ]:
!rm -r lightning_logs

In [ ]:
trainer.fit(model, data_module)

In [ ]:
trained_model = NERModel.load_from_checkpoint("checkpoints/ner-v4.ckpt")
trained_model.freeze()

In [ ]:
from src.t5.utils import evaluate_f1, generate_answer_batched

predictions = generate_answer_batched(
    trained_model=trained_model.model, tokenizer=tokenizer, data=test_df, batch_size=512
)

In [ ]:
ldf = test_df.copy()
ldf["predictions"] = predictions
ldf["tcomp"] = ldf["target_text"].str.split(";", expand=True)[0]
ldf["pcomp"] = ldf["predictions"].str.split(";", expand=True)[0]

In [ ]:
# [pin]

evaluate_f1(
    predictions=ldf["pcomp"].tolist(),
    labels=ldf["tcomp"].tolist(),
)

0.6451461823244122